## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Fairbanks,US,64.8378,-147.7164,26.01,33,1,9.22,clear sky
1,1,Saint-Philippe,RE,-21.3585,55.7679,77.00,73,0,5.75,clear sky
2,2,Boffa,GN,10.1667,-14.0333,77.32,83,56,6.44,broken clouds
3,3,Lompoc,US,34.6391,-120.4579,64.00,63,86,18.41,overcast clouds
4,4,Rikitea,PF,-23.1203,-134.9692,78.35,74,8,8.57,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 50


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
17,17,Idritsa,RU,56.3316,28.8988,45.03,89,85,6.35,overcast clouds
29,29,Tongliao,CN,43.6125,122.2653,49.71,52,9,21.99,clear sky
31,31,Pokrovskoye,RU,47.4157,38.8971,46.99,100,100,11.01,overcast clouds
33,33,Hasaki,JP,35.7333,140.8333,46.40,93,20,4.61,few clouds
38,38,Bluff,NZ,-46.6000,168.3333,50.00,98,22,1.99,light rain
45,45,Mataura,NZ,-46.1927,168.8643,50.00,98,82,1.99,broken clouds
48,48,Ushuaia,AR,-54.8000,-68.3000,50.00,53,75,13.80,broken clouds
53,53,New Norfolk,AU,-42.7826,147.0587,48.99,80,100,2.37,overcast clouds
75,75,Rovnoye,RU,50.7765,46.0509,49.44,86,99,8.79,overcast clouds
94,94,Kimry,RU,56.8667,37.3500,48.16,74,95,5.64,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                55
City                   55
Country                55
Lat                    55
Lng                    55
Max Temp               55
Humidity               55
Cloudiness             55
Wind Speed             55
Current Description    55
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
17,Idritsa,RU,45.03,overcast clouds,56.3316,28.8988,
29,Tongliao,CN,49.71,clear sky,43.6125,122.2653,
31,Pokrovskoye,RU,46.99,overcast clouds,47.4157,38.8971,
33,Hasaki,JP,46.40,few clouds,35.7333,140.8333,
38,Bluff,NZ,50.00,light rain,-46.6000,168.3333,
45,Mataura,NZ,50.00,broken clouds,-46.1927,168.8643,
48,Ushuaia,AR,50.00,broken clouds,-54.8000,-68.3000,
53,New Norfolk,AU,48.99,overcast clouds,-42.7826,147.0587,
75,Rovnoye,RU,49.44,overcast clouds,50.7765,46.0509,
94,Kimry,RU,48.16,overcast clouds,56.8667,37.3500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'].astype(bool)]

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))